In [ ]:
#ex 1a
from pgmpy.models import DiscreteBayesianNetwork

model = DiscreteBayesianNetwork([('S', 'O'), ('S', 'L'), ('S', 'M'), ('L', 'M')])
ind = model.get_independencies()
print(ind)

#b

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([('S', 'O'), ('S', 'L'), ('S', 'M'), ('L', 'M')])

cpd_s = TabularCPD(variable='S', variable_card=2, values=[[0.6], [0.4]])
cpd_o = TabularCPD(variable='O', variable_card=2,
                   values=[[0.3, 0.9],
                           [0.7, 0.1]],
                   evidence=['S'], evidence_card=[2])
cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.2, 0.7],
                           [0.8, 0.3]],
                   evidence=['S'], evidence_card=[2])
cpd_m = TabularCPD(variable='M', variable_card=2,
    values=[ [0.8, 0.4, 0.5, 0.1],
        [0.2, 0.6, 0.5, 0.9]],
    evidence=['S', 'L'],
    evidence_card=[2, 2]
)


model.add_cpds(cpd_s, cpd_o, cpd_l, cpd_m)

print(model.check_model())

inference = VariableElimination(model)
posterior = inference.query(variables=['S'], evidence={'O': 1, 'L': 1, 'M': 1})
print(posterior)

(O ⟂ M | S)
(O ⟂ L | S)
True
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.9492 |
+------+----------+
| S(1) |   0.0508 |
+------+----------+


In [ ]:
#ex 2:
#ex1 lab2
import random
def ex1():
    r=3
    b=4
    bl=2
    z=random.randint(1,6)
    if z==2 or z==3 or z==5:
        bl=bl+1
    elif z==6:
        r=r+1
    else:
        b=b+1

    total=["red"]*r+["blue"]*b+["black"]*bl
    return random.choice(total)

#b

suma=0
for i in range(12000):
    if ex1()=="red":
        suma=suma+1

print(suma/12000)

#cu bayesian

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([('Z', 'Color')])

cpd_z = TabularCPD(variable='Z', variable_card=6,
                   values=[[1/6]] * 6)

def compute_probs(r, b, bl):
    total = r + b + bl
    return [r/total, b/total, bl/total]

color_probs = [
    compute_probs(3,5,2),
    compute_probs(3,4,3),
    compute_probs(3,4,3),
    compute_probs(3,5,2),
    compute_probs(3,4,3),
    compute_probs(4,4,2),
]

color_probs_T = list(map(list, zip(*color_probs)))

cpd_color = TabularCPD(variable='Color', variable_card=3,
                       values=color_probs_T,
                       evidence=['Z'],
                       evidence_card=[6])

model.add_cpds(cpd_z, cpd_color)
assert model.check_model()

inference = VariableElimination(model)
posterior = inference.query(variables=['Color'])
print(posterior)

bn_red_prob = posterior.values[0]

print(f"Bayesian {bn_red_prob:.4f}")
print(f"No Bayesian {suma/12000:.4f}")


0.31808333333333333
+----------+--------------+
| Color    |   phi(Color) |
+==========+==============+
| Color(0) |       0.3167 |
+----------+--------------+
| Color(1) |       0.4333 |
+----------+--------------+
| Color(2) |       0.2500 |
+----------+--------------+
Bayesian 0.3167
No Bayesian 0.3181


In [ ]:
#ex3
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np
from scipy.stats import binom

model = DiscreteBayesianNetwork([
    ('Starter', 'N'),
    ('Starter', 'Flipper'),
    ('N', 'M'),
    ('Flipper', 'M'),
    ('Starter', 'Winner'),
    ('N', 'Winner'),
    ('M', 'Winner')
])

cpd_starter = TabularCPD('Starter', 2, [[0.5], [0.5]])

values_n = np.array([[1/6]*2]*6)
cpd_n = TabularCPD('N', 6, values=values_n, evidence=['Starter'], evidence_card=[2])

cpd_flipper = TabularCPD('Flipper', 2,
                         values=[[0,1],
                                 [1,0]],
                         evidence=['Starter'], evidence_card=[2])

variable_card_M = 13
cpd_m_values = np.zeros((variable_card_M, 6*2))

for n_idx in range(6):
    n = n_idx + 1
    for f_idx in range(2):
        p_heads = 0.5 if f_idx == 0 else 4/7
        flips = 2 * n
        pmf = np.array([binom.pmf(k, flips, p_heads) if k <= flips else 0 for k in range(variable_card_M)])
        col_idx = n_idx*2 + f_idx
        cpd_m_values[:, col_idx] = pmf

cpd_m = TabularCPD('M', variable_card_M, cpd_m_values,
                   evidence=['N', 'Flipper'], evidence_card=[6,2])

cpd_winner_values = np.zeros((2, 2*6*variable_card_M))

for starter in range(2):
    for n_idx in range(6):
        n = n_idx + 1
        for m in range(variable_card_M):
            col_idx = starter*6*variable_card_M + n_idx*variable_card_M + m
            if n >= m:
                cpd_winner_values[0, col_idx] = 1
            else:
                cpd_winner_values[1, col_idx] = 1

cpd_winner = TabularCPD('Winner', 2, cpd_winner_values,
                        evidence=['Starter', 'N', 'M'],
                        evidence_card=[2, 6, variable_card_M])

model.add_cpds(cpd_starter, cpd_n, cpd_flipper, cpd_m, cpd_winner)
assert model.check_model()

inference = VariableElimination(model)
posterior_starter = inference.query(variables=['Starter'], evidence={'M': 1})

print(posterior_starter)


+------------+----------------+
| Starter    |   phi(Starter) |
+============+================+
| Starter(0) |         0.4529 |
+------------+----------------+
| Starter(1) |         0.5471 |
+------------+----------------+
